### Проверка того, что наше api работает

Перед началом необходимо собрать docker-образ и запустить контейнер

1. Следовать инструкциям https://github.com/fimochka-sudo/GB_docker_flask_example/blob/main/README.md
2. Далее уже запускать код ниже

In [15]:
import pandas as pd
from urllib import request, parse
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, precision_score

In [6]:
comments = pd.read_csv("mbti_1.csv")
X_train, X_test, y_train, y_test = train_test_split(comments, comments['type'], random_state=0)
X_test.head(3)

,type,posts
4587,ISFP,"'Dear ISFJ Mother, I wish you were less of a w..."
2786,INFJ,"'To me, I think you guys may be over analyzing..."
2813,ENFP,"'NIHM While NIHM has her INTJ husband, I've go..."


In [7]:
import urllib.request
import json      


def get_prediction(posts):
    body = {'posts': posts} 
    myurl = "http://localhost:8180/predict"
    req = urllib.request.Request(myurl)
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    jsondata = json.dumps(body)
    jsondataasbytes = jsondata.encode('utf-8')   # needs to be bytes
    req.add_header('Content-Length', len(jsondataasbytes))
    #print (jsondataasbytes)
    response = urllib.request.urlopen(req, jsondataasbytes)
    return json.loads(response.read())['predictions']

In [20]:
get_prediction('hello darkness my old friend')

'INTP'

In [19]:
X_test['posts'][2786]

"'To me, I think you guys may be over analyzing it a bit. I know I do this often with friends, whether I mean to or not. I even usually realize that I do it and can't help myself. So since this sounds...|||I definitely do not think it is just an ENTP thing. I am INFJ and have refused to jump through hoops for anyone for most of my life. I was raised to follow all of these stupid little rules of social...|||For one of the millions of applications I have submitted to get recognized and to get hired. And hot chocolate in front of a giant bonfire.|||Hmm, I have never heard of that, but it does make sense. Thanks for posting.|||I tend to go through these periods too. It seems like I keep going back and forth between two phases, wanting to be close to people and feeling extremely lonely and the other phase is where I tend to...|||I have to completely agree with her also. She hit it dead on the nose. INFxs tend to have strong emotions, but we do not display them as much as you would think. Ju

In [18]:
X_test['posts'].iloc[:500]

4587    'Dear ISFJ Mother, I wish you were less of a w...
2786    'To me, I think you guys may be over analyzing...
2813    'NIHM While NIHM has her INTJ husband, I've go...
3705    'I want 5 kids: - an astro/nuclear/theoretical...
5957    'I have the same thing as well. I've noticed t...
                              ...                        
4493    'lol i think i can fall in love with every typ...
6543    'icicle84   The hymns are really beautiful!! B...
8271    'I have absolutely no idea who Lemons is. Just...
2026    'I'm loud, I speak fast, and I am an INTJ. So....
6346    'maybe the things you observed on yourself are...
Name: posts, Length: 500, dtype: object

In [12]:
%%time
predictions = X_test['posts'].iloc[:500].apply(lambda x: get_prediction(x), 1)

CPU times: user 499 ms, sys: 147 ms, total: 646 ms
Wall time: 25.2 s


In [13]:
predictions

4587    INTP
2786    INFJ
2813    ENFP
3705    INTJ
5957    ENFP
        ... 
4493    ENFP
6543    INFP
8271    ENFP
2026    INFP
6346    INFP
Name: posts, Length: 500, dtype: object

In [16]:
print(f1_score(y_test.iloc[:500], predictions, average='micro'))
print(f1_score(y_test.iloc[:500], predictions, average='macro'))
print(f1_score(y_test.iloc[:500], predictions, average='weighted'))
print(precision_score(y_test.iloc[:500], predictions, average='micro'))
print(precision_score(y_test.iloc[:500], predictions, average='macro'))
print(precision_score(y_test.iloc[:500], predictions, average='weighted'))

0.428
0.2417903925300635
0.41606693400259787
0.428
0.2751201758675218
0.4301062462333739
